In [21]:
text_fr = """Il était une fois, dans la ville de Paris, une jeune femme nommée Marie qui travaillait pour l'Organisation des Nations unies pour l'alimentation et l'agriculture (FAO). Elle avait pour mission de se rendre dans différents pays pour étudier les pratiques agricoles locales et promouvoir des méthodes durables pour nourrir la population mondiale croissante. Marie était passionnée par son travail et avait déjà visité de nombreux endroits, tels que le Brésil, le Kenya et l'Inde.

Un jour, alors qu'elle se trouvait en mission en Égypte, Marie rencontra un homme charmant du nom d'Ahmed. Il travaillait pour une organisation locale appelée l'Association égyptienne pour le développement rural (AEDR), qui avait pour objectif d'améliorer les conditions de vie des communautés rurales en Égypte. Ahmed était également très intéressé par l'agriculture durable et lui et Marie se lièrent d'amitié rapidement.

Marie et Ahmed ont continué à travailler ensemble pour promouvoir des pratiques agricoles durables en Égypte et ont finalement créé une organisation conjointe appelée l'Initiative pour une agriculture durable (IAS). Ils ont réussi à obtenir un financement de l'Union européenne pour leur projet et ont pu étendre leur travail à d'autres pays du Moyen-Orient.

Au fil des années, l'IAS est devenue une organisation de premier plan dans le domaine de l'agriculture durable et a reçu de nombreux prix pour son travail. Marie et Ahmed ont continué à diriger l'organisation ensemble et ont fondé une famille heureuse en Égypte. Leur travail a permis d'améliorer la vie de nombreuses personnes dans le monde entier et leur héritage continue de vivre à travers l'IAS, qui continue à promouvoir des pratiques agricoles durables et à lutter contre la faim dans le monde."""

In [22]:
!pip install transformers

In [23]:
!pip install sentencepiece

## 1. NER SpaCy Model

We will start by trying NER model proposed by SpaCy

In [24]:
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer import AnalyzerEngine, EntityRecognizer, RecognizerResult, Pattern, PatternRecognizer

from presidio_analyzer.nlp_engine import NlpArtifacts,NlpEngineProvider


In [25]:
configuration = {"nlp_engine_name":"spacy", "models":[{"lang_code":"fr", "model_name":"fr_core_news_lg"}]}


provider = NlpEngineProvider(nlp_configuration=configuration)

nlp_engine = provider.create_engine()


analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine,
    supported_languages = ['fr']
)

In [26]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
# Loading both tokenizer and NER model
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
ner_model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

## Mixed Pipeline development (Transformers + SpaCy)

In [27]:
from transformers import pipeline
# list of entities: https://microsoft.github.io/presidio/supported_entities/#list-of-supported-entities
DEFAULT_ANOYNM_ENTITIES = [
    "CREDIT_CARD",
    "CRYPTO",
    "DATE_TIME",
    "EMAIL_ADDRESS",
    "IBAN_CODE",
    "IP_ADDRESS",
    "NRP",
    "LOCATION",
    "PERSON",
    "PHONE_NUMBER",
    "MEDICAL_LICENSE",
    "URL",
    "ORGANIZATION",
    "NUMBER"
]

class TransformerRecognizer(EntityRecognizer):
    def __init__(
        self,
        model_id_or_path,
        mapping_labels,
        aggregation_strategy="simple",
        supported_language="fr",
        ignore_labels=["O", "MISC"],
    ):
        # inits transformers pipeline for given mode or path
        self.pipeline = pipeline(
            "token-classification", model=model_id_or_path, aggregation_strategy=aggregation_strategy, ignore_labels=ignore_labels
        )
        # map labels to presidio labels
        self.label2presidio = mapping_labels

        # passes entities from model into parent class
        super().__init__(supported_entities=list(self.label2presidio.values()), supported_language=supported_language)

    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(
        self, text: str, entities = None, nlp_artifacts: NlpArtifacts = None
    ):
        """
        Extracts entities using Transformers pipeline
        """
        results = []

        predicted_entities = self.pipeline(text)
        if len(predicted_entities) > 0:
            for e in predicted_entities:
                if(e['entity_group'] not in self.label2presidio):
                    continue
                converted_entity = self.label2presidio[e["entity_group"]]
                if converted_entity in entities or entities is None:
                    results.append(
                        RecognizerResult(
                            entity_type=converted_entity, start=e["start"], end=e["end"], score=e["score"]
                        )
                    )
        return results

In [28]:
#mapping_labels = {"PROPN": "PERSON","XFAMIL": "PERSON"}
mapping_labels = {"PER":"PERSON",'LOC':'LOCATION','ORG':"ORGANIZATION",'PHONE_NUMBER':'PHONE_NUMBER'}
configuration = {"nlp_engine_name":"spacy",
                "models":[{"lang_code": 'fr', "model_name":"fr_core_news_lg"}]}


to_keep = []
lang = 'fr'

In [29]:
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine,
    supported_languages = "fr"
)

transformers_recognizer = TransformerRecognizer("Jean-Baptiste/camembert-ner", mapping_labels)
analyzer.registry.add_recognizer(transformers_recognizer)

In [35]:
# Text Analyzer
analyzer_results = analyzer.analyze(text=text_fr, entities = DEFAULT_ANOYNM_ENTITIES, allow_list = to_keep, language=lang)

# Text Anonymizer
engine = AnonymizerEngine()
result = engine.anonymize(text=text_fr, analyzer_results=analyzer_results)

# Restructuring anonymizer results

anonymization_results =  {"anonymized": result.text,"found": [entity.to_dict() for entity in analyzer_results]}

words = [{'word': text_fr[obj['start']:obj['end']], 'entity_type':obj['entity_type'], 'start':obj['start'], 'end':obj['end']} for obj in anonymization_results['found']]

In [36]:
words

[{'word': ' Kenya', 'entity_type': 'LOCATION', 'start': 461, 'end': 467},
 {'word': 'Inde', 'entity_type': 'LOCATION', 'start': 473, 'end': 477},
 {'word': ' Brésil', 'entity_type': 'LOCATION', 'start': 450, 'end': 457},
 {'word': ' Moyen-Orient',
  'entity_type': 'LOCATION',
  'start': 1249,
  'end': 1262},
 {'word': ' Égypte', 'entity_type': 'LOCATION', 'start': 528, 'end': 535},
 {'word': ' Égypte', 'entity_type': 'LOCATION', 'start': 1519, 'end': 1526},
 {'word': ' Égypte', 'entity_type': 'LOCATION', 'start': 784, 'end': 791},
 {'word': 'FAO', 'entity_type': 'ORGANIZATION', 'start': 164, 'end': 167},
 {'word': ' Égypte', 'entity_type': 'LOCATION', 'start': 1006, 'end': 1013},
 {'word': 'IAS', 'entity_type': 'ORGANIZATION', 'start': 1661, 'end': 1664},
 {'word': 'IAS', 'entity_type': 'ORGANIZATION', 'start': 1286, 'end': 1289},
 {'word': 'Union européenne',
  'entity_type': 'ORGANIZATION',
  'start': 1166,
  'end': 1182},
 {'word': ' Paris', 'entity_type': 'LOCATION', 'start': 35, '

In [37]:
# We had to strip the results to remove the leading spaces and \n
word_results = [res['word'].strip() for res in words]

In [38]:
set(word_results)

{'AEDR',
 'Ahmed',
 'Association égyptienne pour le développement rural',
 'Brésil',
 'FAO',
 'IAS',
 'Inde',
 'Initiative pour une agriculture durable',
 'Kenya',
 'Marie',
 'Moyen-Orient',
 "Organisation des Nations unies pour l'alimentation et l'agriculture",
 'Paris',
 'Union européenne',
 'Égypte'}

In [39]:
anonymization_results['anonymized']

"Il était une fois, dans la ville de<LOCATION>, une jeune femme nommée<PERSON> qui travaillait pour l'<ORGANIZATION> (<ORGANIZATION>). Elle avait pour mission de se rendre dans différents pays pour étudier les pratiques agricoles locales et promouvoir des méthodes durables pour nourrir la population mondiale croissante.<PERSON> était passionnée par son travail et avait déjà visité de nombreux endroits, tels que le<LOCATION>, le<LOCATION> et l'<LOCATION>.\n\nUn jour, alors qu'elle se trouvait en mission en<LOCATION>,<PERSON> rencontra un homme charmant du nom d'<PERSON>. Il travaillait pour une organisation locale appelée l'<ORGANIZATION> (<ORGANIZATION>), qui avait pour objectif d'améliorer les conditions de vie des communautés rurales en<LOCATION>.<PERSON> était également très intéressé par l'agriculture durable et lui et<PERSON> se lièrent d'amitié rapidement.\n<PERSON> et<PERSON> ont continué à travailler ensemble pour promouvoir des pratiques agricoles durables en<LOCATION> et ont 